In [1]:
import requests
from bs4 import BeautifulSoup
import pandas

In [2]:
account = '106010'
password = 'G122117609'
session = requests.session()

In [3]:
def get_session_login(account, password, session):
    login_url = 'http://202.3.168.17:8080/login_check.jsp'
    login_data = {
        'sess_id': account,
        'sess_password': password,
        'dfURL': '',
        'dfFrame_no': 2,
        'dfAction': '',
        'dfId': '',
        'dfForm': 'login.jsp'
    }
    session.post(url=login_url, data=login_data)
    return session

In [4]:
session = get_session_login(account=account, password=password, session=session)

In [5]:
def get_df_handling_ticket(session):
    dashboard_terminal_url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal.jsp'
    response_dashboard_terminal = session.get(url=dashboard_terminal_url)
    df_read_html_response_dashboard_terminal_text = pandas.read_html(response_dashboard_terminal.text)
    df_handling_ticket = df_read_html_response_dashboard_terminal_text[4]
    return df_handling_ticket

In [6]:
df_handling_ticket = get_df_handling_ticket(session)

In [7]:
def let_the_first_row_be_column_title(dataframe):
    new_dataframe = dataframe[1:]
    new_dataframe.columns = dataframe.iloc[0]
    return new_dataframe

In [8]:
df_handling_ticket_with_title = let_the_first_row_be_column_title(df_handling_ticket)

In [9]:
handling_ticket_number_list = list(df_handling_ticket_with_title['單號'])

In [47]:
def get_df_ticket_detail(session, account, ticket_number):
    ticket_detail_url = 'http://202.3.168.17:8080/Disp/retriveDetail.jsp'
    ticket_detail_data = {
        'method': 'get_Disp_DetailCons',
        'Disp_Cons_Seq': account,
        'Disp_Seq': ticket_number,
    }
    response_ticket_detail = session.post(url=ticket_detail_url, data=ticket_detail_data)
    df_read_html_response_ticket_detail_text = pandas.read_html(response_ticket_detail.text)
    df_ticket_detail = df_read_html_response_ticket_detail_text[0]
    return df_ticket_detail

In [53]:
df_list = []
for ticket_number in handling_ticket_number_list:
    df_ticket_detail = get_df_ticket_detail(session=session, account=account, ticket_number=ticket_number)
    df_ticket_detail_with_title = let_the_first_row_be_column_title(df_ticket_detail)
    df_ticket_detail_with_title_with_ticket_number = df_ticket_detail_with_title.assign(單號=ticket_number)
    df_list += [df_ticket_detail_with_title_with_ticket_number]

In [56]:
df_ticket_detail_total = pandas.concat(df_list)

In [58]:
df_left = df_ticket_detail_total
df_right = df_handling_ticket_with_title.drop(columns=['流程'])
result = pandas.merge(df_left, df_right, on='單號', how='left')

In [62]:
text_file = open('practice.html', "w")
text_file.write(result.to_html())
text_file.close()